In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
data = pd.read_parquet(
    "../../data/2_clean/education_attainment.parquet",
)

In [3]:
data_stacked = (
    data.set_index(
        [
            "Pays et territoires insulaires du Pacifique",
            "TIME_PERIOD",
            "Sexe",
        ]
    )
    .drop(columns=["Âge", "Urbanisation", "Invalidité"])
    .stack()
    .reset_index()
    .rename(columns={0: "Value"})
)
data_stacked = data_stacked.query("Sexe != 'Total'")

level_cat = {
    "L'éducation de la petite enfance": "Petite enfance",
    "Enseignement primaire": "Primaire",
    "Enseignement secondaire inférieur": "Secondaire inf.",
    "L'enseignement secondaire supérieur": "Secondaire sup.",
    "Enseignement post-secondaire non supérieur": "Post-sec. non sup.",
    "Éducation tertiaire": "Tertiaire",
    "Niveau non indiqué": "Non indiqué",
    "Tous les niveaux d'enseignement": "Total",
}

data_stacked["Niveau d'éducation"] = pd.Categorical(
    data_stacked["Niveau d'éducation"].replace(level_cat),
    ordered=True,
    categories=level_cat.values(),
)

data_stacked = data_stacked.sort_values(
    [
        "Pays et territoires insulaires du Pacifique",
        "TIME_PERIOD",
        "Sexe",
        "Niveau d'éducation",
    ],
)
data_stacked

,Pays et territoires insulaires du Pacifique,TIME_PERIOD,Sexe,Niveau d'éducation,Value
4,Kiribati,2020,Femme,Petite enfance,384.0
12,Kiribati,2020,Femme,Petite enfance,86.0
20,Kiribati,2020,Femme,Petite enfance,298.0
28,Kiribati,2020,Femme,Petite enfance,157.0
35,Kiribati,2020,Femme,Petite enfance,50.0
...,...,...,...,...,...
3986,Vanuatu,2020,Homme,Total,2636.0
3993,Vanuatu,2020,Homme,Total,45079.0
4000,Vanuatu,2020,Homme,Total,15456.0
4006,Vanuatu,2020,Homme,Total,663.0


## KIRIBATI


In [4]:
kiribati = data_stacked[
    data_stacked["Pays et territoires insulaires du Pacifique"] == "Kiribati"
]
kiribati = (
    kiribati.drop(columns="Pays et territoires insulaires du Pacifique")
    .groupby(["TIME_PERIOD", "Sexe", "Niveau d'éducation"])
    .sum()
    .replace(0, np.nan)
    .dropna()
)
kiribati

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_4751/1603260062.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["TIME_PERIOD", "Sexe", "Niveau d'éducation"])


Value
TIME_PERIOD Sexe  Niveau d'éducation          
2020        Femme Petite enfance        7872.0
                  Primaire             26728.0
                  Secondaire inf.      83552.0
                  Secondaire sup.      93224.0
                  Post-sec. non sup.    3963.0
                  Tertiaire             7922.0
                  Non indiqué           4290.0
                  Total               227584.0
            Homme Petite enfance        6560.0
                  Primaire             25032.0
                  Secondaire inf.      80416.0
                  Secondaire sup.      73336.0
                  Post-sec. non sup.    3658.0
                  Tertiaire             6805.0
                  Non indiqué           4017.0
                  Total               199856.0

In [5]:
fig = px.bar(
    kiribati.reset_index(),
    x="TIME_PERIOD",
    y="Value",
    color="Niveau d'éducation",
    facet_row="Sexe",
    barmode="group",
    title="Education attainment in Kiribati",
    labels={"Value": "Total population"},
    range_y=[0, kiribati.max().iloc[0]],
)
fig.show()

In [6]:
kiribati = data_stacked[
    data_stacked["Pays et territoires insulaires du Pacifique"] == "Kiribati"
]
kiribati = (
    kiribati.drop(columns="Pays et territoires insulaires du Pacifique")
    .groupby(["TIME_PERIOD", "Sexe", "Niveau d'éducation"])
    .sum()
    .replace(0, np.nan)
    .dropna()
)
kiribati

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_4751/1603260062.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Value
TIME_PERIOD Sexe  Niveau d'éducation          
2020        Femme Petite enfance        7872.0
                  Primaire             26728.0
                  Secondaire inf.      83552.0
                  Secondaire sup.      93224.0
                  Post-sec. non sup.    3963.0
                  Tertiaire             7922.0
                  Non indiqué           4290.0
                  Total               227584.0
            Homme Petite enfance        6560.0
                  Primaire             25032.0
                  Secondaire inf.      80416.0
                  Secondaire sup.      73336.0
                  Post-sec. non sup.    3658.0
                  Tertiaire             6805.0
                  Non indiqué           4017.0
                  Total               199856.0

## ALL


Each country only have 1 year. So the year column can be skipped.


In [7]:
all_data = (
    data_stacked.groupby(
        ["Pays et territoires insulaires du Pacifique", "Sexe", "Niveau d'éducation"]
    )["Value"]
    .mean()
    .replace(0, np.nan)
    .dropna()
    .reset_index()
)

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_4751/1665159970.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [8]:
fig = px.bar(
    all_data.reset_index(),
    x="Sexe",
    y="Value",
    color="Niveau d'éducation",
    facet_row="Pays et territoires insulaires du Pacifique",
    barmode="group",
    title="Education attainment in Kiribati",
    labels={"Value": "Total population"},
    # range_y=[0, all_data["Value"].max()],
    # Cahange the height of the figure
    height=200 * len(all_data["Pays et territoires insulaires du Pacifique"].unique()),
)
# don't share X between rows
fig.update_yaxes(matches=None)
# don't show "Pays et territoires insulaires du Pacifique=" on the right of the plot
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# show x axis labels on the top of the first row
for i, row in enumerate(fig.select_traces()):
    # show x labels
    fig.update_xaxes(
        row=i + 1,
        showticklabels=True,
        title_text="",
    )
fig
fig.show()

## Ratio all


In [9]:
ratio_all_homme = (
    all_data.query("Sexe == 'Homme'")
    .drop(columns="Sexe")
    .groupby(["Pays et territoires insulaires du Pacifique", "Niveau d'éducation"])
    .sum()
)
ratio_all_femme = (
    all_data.query("Sexe == 'Femme'")
    .drop(columns="Sexe")
    .groupby(["Pays et territoires insulaires du Pacifique", "Niveau d'éducation"])
    .sum()
)
ratio_all = ratio_all_homme + ratio_all_femme

ratio_all_homme = (ratio_all_homme / ratio_all) * 100
ratio_all_femme = (ratio_all_femme / ratio_all) * 100

ratio = ratio_all_homme.copy().rename(columns={"Value": "Homme"})
ratio["Femme"] = ratio_all_femme["Value"]
ratio = ratio.stack().rename("Ratio").reset_index().rename(columns={"level_2": "Genre"})
ratio

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_4751/1741038620.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_4751/1741038620.py:10: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Pays et territoires insulaires du Pacifique,Niveau d'éducation,Genre,Ratio
0,Kiribati,Petite enfance,Homme,45.454545
1,Kiribati,Petite enfance,Femme,54.545455
2,Kiribati,Primaire,Homme,48.361669
3,Kiribati,Primaire,Femme,51.638331
4,Kiribati,Secondaire inf.,Homme,49.043716
...,...,...,...,...
99,Vanuatu,Tertiaire,Femme,40.871031
100,Vanuatu,Non indiqué,Homme,56.791569
101,Vanuatu,Non indiqué,Femme,43.208431
102,Vanuatu,Total,Homme,49.187625


In [10]:
fig = px.bar(
    ratio,
    x="Niveau d'éducation",
    y="Ratio",
    color="Genre",
    facet_row="Pays et territoires insulaires du Pacifique",
    barmode="stack",
    title="Répartition des niveaux d'éducation par genre et par territoire",
    labels={"Value": "Total population"},
    range_y=[0, 100],
    height=200 * len(all_data["Pays et territoires insulaires du Pacifique"].unique()),
    width=800,
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [11]:
ratio.to_parquet("../../data/3_product/education_attainment.parquet", index=False)